# Run multiple rainfall events

In [ ]:
from pathlib import Path
import mikeio
import mikeplus as mp

rainfall_timeseries = Path("../tests/testdata/Db/Sirius/Sirius_IDF_rainfall.dfs0")
db_path = Path("../tests/testdata//Db/Sirius/Sirius.sqlite")

#### Explore rainfall boundary dfs0 file

In [ ]:
ds = mikeio.read(rainfall_timeseries)
ds

In [ ]:
event_info = []
for item in ds.items:
    event_info.append(
        {
            "file_path": str(rainfall_timeseries.relative_to(db_path.parent)),
            "item_name": item.name,
            "data_type": repr(item.type),
        }
    )

event_info

#### Create boundary conditions for each rainfall event

In [ ]:
db = mp.open(db_path)
db

In [ ]:
# No boundary conditions currently exist
msm_BBoundary = db.tables.msm_BBoundary
msm_BBoundary.get_muids()

In [ ]:
# Create a boundary condition for each event
msm_BBoundary = db.tables.msm_BBoundary
for event in event_info:
    msm_BBoundary.insert({
        msm_BBoundary.columns.MUID              : "Rainfall " + event["item_name"],
        msm_BBoundary.columns.VariationNo       : 3, # set temporal variation to timeseries
        msm_BBoundary.columns.FlowTypeNo        : 5, # set boundary condition type to Rainfall
        msm_BBoundary.columns.LoadTypeNo        : 1, # set load type to stormwater runoff
        msm_BBoundary.columns.TSConnection      : event["file_path"],
        msm_BBoundary.columns.TimeseriesName    : event["item_name"],
        msm_BBoundary.columns.DataTypeName      : event["data_type"],
        msm_BBoundary.columns.ApplyBoundaryNo   : 0,
    })

In [ ]:
# Verify
def view_boundaries():
    return msm_BBoundary.select([
        msm_BBoundary.columns.VariationNo,
        msm_BBoundary.columns.FlowTypeNo,
        msm_BBoundary.columns.LoadTypeNo,
        msm_BBoundary.columns.TSConnection,
        msm_BBoundary.columns.TimeseriesName,
        msm_BBoundary.columns.DataTypeName,
        msm_BBoundary.columns.ApplyBoundaryNo,
    ]).to_dataframe()
view_boundaries()

#### Create a scenario with a corresponding alternative for each rainfall

In [ ]:
# We are in the base scenario
db.active_scenario

In [ ]:
# We'll create altenatives under the Loads and boundaries data alternative group
db.alternative_groups["Loads and boundaries data"].tables

In [ ]:
# The alternative for the active scenario
db.alternative_groups["Loads and boundaries data"].active

In [ ]:
for boundary in msm_BBoundary.get_muids():

    # Create scenario
    scenario = db.scenarios.create(name=boundary, parent=db.active_scenario)

    # Create alternative
    alternative = db.alternative_groups["Loads and boundaries data"].create(
        name=boundary,
        parent=db.alternative_groups["Loads and boundaries data"].active
    )
    
    # Set alternative for new scenario
    scenario.set_alternative(alternative)

    # Activate the new scenario and update boundary
    db.active_scenario = scenario
    updated_muids = (
        msm_BBoundary
        .update({
            msm_BBoundary.columns.ApplyBoundaryNo: 1
        })
        .where(f"MUID = '{boundary}'")
        .execute() # don't forget to execute!
    )
    print(f"Updated boundary MUIDs: {updated_muids}")

    # Reset back to base scenario for next iteration
    db.active_scenario = db.scenarios.base


In [ ]:
# The base scenario should still have no boundaries applied
db.active_scenario = db.scenarios.base
view_boundaries()

In [ ]:
# Verify boundary is applied
db.active_scenario = db.scenarios.by_name("Rainfall  F=20")
view_boundaries()

In [ ]:
# Verify boundary is applied
db.active_scenario = db.scenarios.by_name("Rainfall  F=0.05")
view_boundaries()

#### Run each event

In [ ]:
# One existing simulation will be used
db.tables.msm_Project.select().to_dataframe()

In [ ]:
# Get the scenarios with "Rainfall" in the name
scenarios = [scenario for scenario in db.scenarios if "Rainfall" in scenario.name]
scenarios

In [ ]:
# Run each scenario
results = {}

for scenario in scenarios:
    updated_muids = (
        db.tables.msm_Project
        .update({
            db.tables.msm_Project.columns.ScenarioName : scenario.name
        })
        .where(f"MUID = 'Sirius_1_DEMO'")
        .execute()
    )

    if not updated_muids:
        raise ValueError(f"Failed to update scenario '{scenario.name}'.")

    result_files = db.run(simulation_muid="Sirius_1_DEMO")
    result_files = [f.name for f in result_files]           # hide full path (optional)
    results[scenario.name] = result_files

results

In [ ]:
db.close()